In [1]:
import os
import sys

def add_to_path(str):
    try:
        sys.path.index(str)
    except:
        print(str + "\t is already in the path list")
    else:
        sys.path.append(str)
        print(str + "\t add to the path list")

# this add all the libraris and scripts made for this project

add_to_path('../datalib/')
add_to_path('../khinsider-master/')

In [4]:
import wave
import numpy as np
import scipy as sp

from matplotlib import pyplot as plt

import pprint
import struct

import pandas as pd

import time

# Part 1 : Information Retrive

## * Getting Sound Samples
In this case, despite the main idea was getting already noici samples, it was need change this, because the "master" sample needs to be almost pure , whit the minimal noise posible

In [5]:
# Download
# os.system("python3.6 ../khinsider-master/khinsider.py half-lifel-2-official-soundtrack")

In [6]:
DATA_PATH = "../data/"
files_wav = [ file for file in os.listdir(DATA_PATH) if file[-4:] == ".wav"]

In [7]:
files_wav

['01 - tank! (tv stretch).wav',
 '04 - diamonds.wav',
 '07 - mushroom hunting.wav',
 "10 - einstein groovin'.wav",
 '08 - no reply.wav',
 '06 - piano black.wav',
 '03 - cosmic dare (pretty with a pistol).wav',
 '11 - pearls.wav',
 '09 - blue.wav',
 '12 - gotta knock a little harder.wav',
 '02 - what planet is this..wav',
 "05 - don't bother none (tv edit).wav"]

In [8]:
index = 0
sound_data_container = {}
for wav_f in files_wav:
    sound_data_container[index] = [wave.open(DATA_PATH + wav_f), wav_f]
    print(index , wav_f, "\n")
    index += 1

print(sound_data_container.__sizeof__())

0 01 - tank! (tv stretch).wav 

1 04 - diamonds.wav 

2 07 - mushroom hunting.wav 

3 10 - einstein groovin'.wav 

4 08 - no reply.wav 

5 06 - piano black.wav 

6 03 - cosmic dare (pretty with a pistol).wav 

7 11 - pearls.wav 

8 09 - blue.wav 

9 12 - gotta knock a little harder.wav 

10 02 - what planet is this..wav 

11 05 - don't bother none (tv edit).wav 

624


In [9]:
def stereo_chanel_data(wav_obj_pointer, verbose=False):
    wav_obj_pointer.rewind()
    
    signed_data_l = []
    signed_data_r = []

    total_frames = wav_obj_pointer.getnframes()
    amount = 0

    for frame_index in range(1 , total_frames - 1, 1):
        wav_obj_pointer.setpos(frame_index)

        try:
            l_int = struct.unpack("<h",wav_obj_pointer.readframes(1)[:2])[0]
            r_int = struct.unpack("<h",wav_obj_pointer.readframes(1)[-2:])[0]

        except:
            if verbose:
                print("error with frame \t" , frame_index , " N_CHANEL_DATA ERROR \n")
        else:
            signed_data_l.append(l_int)
            signed_data_r.append(r_int)

            if frame_index % (44100*20) == 0 and verbose:
                print(round((frame_index / total_frames) * 100, 4), "\t%")
                print(l_int, "\t", r_int)
                print(len(signed_data_l), "\t", len(signed_data_r))

    return [signed_data_l, signed_data_r]

In [10]:
test = sound_data_container[1][0]
print(test.getparams())

_wave_params(nchannels=2, sampwidth=2, framerate=44100, nframes=10653696, comptype='NONE', compname='not compressed')


In [11]:
def sound_csv(files, basic_verbose = False, verbose=False):
    
    for index in range(len(files)):
        
        if verbose:
            print(files[index][1], "\t in execution at : \t 0 % and going")
        elif basic_verbose:
            print(files[index][1], "\t in execution")
        else:
            pass
            
        [sound_obj_ref , sound_name] = files[index]
        
        data_l_r = stereo_chanel_data(sound_obj_ref, verbose)
        
        carry_data = np.matrix(data_l_r).T
        
        sound_frame_data = pd.DataFrame(carry_data, columns=["left", "rigth"])
                
        sound_frame_data.to_csv( (DATA_PATH + sound_name[:-4] + ".csv"), sep='\t', encoding='utf-8')
        
        if verbose or basic_verbose:
            print("\n", sound_name , "\tDONE\n")

In [12]:
start = time.time()
sound_csv(sound_data_container, verbose=True)
print (time.time() - start)

01 - tank! (tv stretch).wav 	 in execution at : 	 0 % and going
9.6511 	%
6687 	 5010
882000 	 882000
19.3023 	%
-635 	 -346
1764000 	 1764000
28.9534 	%
-15969 	 -9634
2646000 	 2646000
38.6046 	%
225 	 4491
3528000 	 3528000
48.2557 	%
-6159 	 -1940
4410000 	 4410000
57.9068 	%
2208 	 3950
5292000 	 5292000
67.558 	%
8673 	 7647
6174000 	 6174000
77.2091 	%
-3032 	 -2280
7056000 	 7056000
86.8603 	%
4329 	 -790
7938000 	 7938000
96.5114 	%
-22352 	 -13344
8820000 	 8820000

 01 - tank! (tv stretch).wav 	DONE

04 - diamonds.wav 	 in execution at : 	 0 % and going
8.2788 	%
787 	 -336
882000 	 882000
16.5576 	%
479 	 -1939
1764000 	 1764000
24.8365 	%
2457 	 2717
2646000 	 2646000
33.1153 	%
6584 	 4401
3528000 	 3528000
41.3941 	%
3353 	 5585
4410000 	 4410000
49.6729 	%
2769 	 -3347
5292000 	 5292000
57.9517 	%
3551 	 3736
6174000 	 6174000
66.2305 	%
1178 	 2073
7056000 	 7056000
74.5094 	%
369 	 -1163
7938000 	 7938000
82.7882 	%
-3064 	 -7029
8820000 	 8820000
91.067 	%
5619 	 501